# segment anything model

In [ ]:

from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"

# device = "mps"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
# sam.to(device=device)

# mask_generator = SamAutomaticMaskGenerator(sam)
mask_generator_2 = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=32,
    pred_iou_thresh=0.86,
    stability_score_thresh=0.92,
    crop_n_layers=1,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=100,  # Requires open-cv to run post-processing
)

def seg_any(image):
    masks_2 = mask_generator_2.generate(image)
    masks_new=[i['segmentation'] for i in masks_2]
    return masks_new

# segmenting

In [ ]:
def generate_segments(image,pth,redo=False):
    if os.path.exists(pth+'_segs'+'.pkl') and redo==False :
        with open(pth+'_segs'+'.pkl', 'rb') as file:
            segs=pickle.load(file)
        print('loaded segments from file')
        return segs

    else:
        print('started segmentation')
        start_time = time.time()
        segs=seg_any(image)
        # segs=_get_segments_felzenszwalb(im,min_size=50) #change here if there is a change in shape
        #segs=_get_segments_felzenszwalb(im,scale_list=[10],min_size=10) #change here if there is a change in shape
        # segs=saliency.xrai._get_segments_felzenszwalb(im,scale_list=[10],min_size=10) #change here if there is a change in shape
        # segs=get_masks()
        end_time = time.time()
        print('time taken by segmentation : '+str(end_time-start_time)+' sec')
        print('completed segmentation')
        with open(pth+'_segs'+'.pkl', 'wb') as file:
            pickle.dump(segs, file)
            file.close()

        return segs


def make_segs_disjoint(segs):
    x=segs[0]*1
    for i in range(1,len(segs)):
        x=x+(segs[i]*(1.01**i))
    new_segs=[]
    for k in np.unique(x):
        new_segs.append(x==k)
    return new_segs

